In [1]:
import re
import string
from underthesea import word_tokenize  # Thay đổi này phụ thuộc vào thư viện bạn sử dụng
import emoji  # Đảm bảo rằng thư viện này được cài đặt
import pandas as pd 
import os
from datetime import datetime, timedelta
import nltk
import statistics 

In [2]:
df1 = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Lazada\Smarphone_Lazada.csv")
df2 = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Shopee\Shopee_Smartphone.csv")
df3 = pd.read_csv(r"D:\projects\NCKH\Data Crawl\Trang TMĐT\Tiki\Code Crawl\Tiki_Smartphone.csv")
df4 = pd.read_csv(r"D:\projects\NCKH\Data_Processing\Smarphones_fpt.csv")
df5 = pd.read_csv(r"D:\projects\NCKH\Data_Processing\Smarphones_TGDĐ.csv")
df6 = pd.read_csv(r"D:\projects\NCKH\Data_Processing\Smarphones_CellPhones.csv")

C:\Users\HELLO\AppData\Local\Temp\ipykernel_11384\2454171215.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv(r"D:\projects\NCKH\Data_Processing\Smarphones_fpt.csv")


In [3]:
print(df1.shape,df2.shape,df3.shape,df4.shape,df5.shape,df6.shape)

(1582, 18) (3126, 19) (274, 21) (224644, 13) (10902, 13) (1769, 14)


# 1. Trích Xuất Dữ Liệu

In [4]:
df1.head(2)

,Platform,URL,Date_Crawl,product_name,brand,current_price,discount_price,discount,rating_count,color,capacity,store,sold_quantity,time_comment,name_cmt,content_cmt,info_cmt,like_count
0,Lazada,https://www.lazada.vn/products/iphone-15-hang-...,3/13/2024,iPhone 15 - Hàng Chính Hãng VN/A,Apple,25.990.000 ₫,24.690.000 ₫,-5%,147 đánh giá,Black,256GB,Apple Flagship Store,707 Đã bán,14 thg 11 2023,T***.,Mua hàng chính hãng tại Apple flagship store t...,"Nhóm Màu:Pink, Dung Lượng Lưu Trữ:128GB",4
1,Lazada,https://www.lazada.vn/products/iphone-15-hang-...,3/13/2024,iPhone 15 - Hàng Chính Hãng VN/A,Apple,25.990.000 ₫,24.690.000 ₫,-5%,147 đánh giá,Black,256GB,Apple Flagship Store,707 Đã bán,02 thg 12 2023,B***h,Chất lượng: \n- Giao hàng nhanh. \n- SP đúng ...,"Nhóm Màu:Xanh dương, Dung Lượng Lưu Trữ:128GB",0


In [5]:
df1s = df1[['Platform','Date_Crawl','brand','product_name','current_price','discount_price','discount','sold_quantity','rating_count','time_comment','content_cmt','info_cmt']].rename(
    columns={
        'rating_count': 'rating_count',
        'time_comment': 'time_comment',
        'sold_quantity': 'sold_quanlity'
    }
)

In [6]:
df1s

,Platform,Date_Crawl,brand,product_name,current_price,discount_price,discount,sold_quanlity,rating_count,time_comment,content_cmt,info_cmt
0,Lazada,3/13/2024,Apple,iPhone 15 - Hàng Chính Hãng VN/A,25.990.000 ₫,24.690.000 ₫,-5%,707 Đã bán,147 đánh giá,14 thg 11 2023,Mua hàng chính hãng tại Apple flagship store t...,"Nhóm Màu:Pink, Dung Lượng Lưu Trữ:128GB"
1,Lazada,3/13/2024,Apple,iPhone 15 - Hàng Chính Hãng VN/A,25.990.000 ₫,24.690.000 ₫,-5%,707 Đã bán,147 đánh giá,02 thg 12 2023,Chất lượng: \n- Giao hàng nhanh. \n- SP đúng ...,"Nhóm Màu:Xanh dương, Dung Lượng Lưu Trữ:128GB"
2,Lazada,3/13/2024,Apple,iPhone 15 - Hàng Chính Hãng VN/A,25.990.000 ₫,24.690.000 ₫,-5%,707 Đã bán,147 đánh giá,17 giờ trước,"Shop chuẩn bị giao hàng nhanh, hình ảnh mang t...","Nhóm Màu:Black, Dung Lượng Lưu Trữ:128GB"
3,Lazada,3/13/2024,Apple,iPhone 15 - Hàng Chính Hãng VN/A,25.990.000 ₫,24.690.000 ₫,-5%,707 Đã bán,147 đánh giá,17 thg 10 2023,"Giao hàng đúng hẹn, chất lượng máy tốt","Nhóm Màu:Màu Xanh Lá Cây, Dung Lượng Lưu Trữ:2..."
4,Lazada,3/13/2024,Apple,iPhone 15 - Hàng Chính Hãng VN/A,25.990.000 ₫,24.690.000 ₫,-5%,707 Đã bán,147 đánh giá,30 thg 9 2023,"Màu hồng cưng xiu, triệu like","Nhóm Màu:Hồng, Dung Lượng Lưu Trữ:256GB"
...,...,...,...,...,...,...,...,...,...,...,...,...
1577,Lazada,3/13/2024,Samsung,Điện thoại Samsung Galaxy S23 Ultra,31.990.000 ₫,27.190.000 ₫,-15%,201,43 đánh giá,21 thg 7 2023,NaN,"Nhóm Màu:Đen, Dung Lượng Lưu Trữ:8GB/256GB"
1578,Lazada,3/13/2024,Samsung,Điện thoại Samsung Galaxy S23 Ultra,31.990.000 ₫,27.190.000 ₫,-15%,201,43 đánh giá,30 thg 7 2023,NaN,"Nhóm Màu:Đen, Dung Lượng Lưu Trữ:8GB/256GB"
1579,Lazada,3/13/2024,Samsung,Điện thoại Samsung Galaxy S23 Ultra,31.990.000 ₫,27.190.000 ₫,-15%,201,43 đánh giá,27 thg 7 2023,NaN,"Nhóm Màu:Kem, Dung Lượng Lưu Trữ:8GB/256GB"
1580,Lazada,3/13/2024,Samsung,Điện thoại Samsung Galaxy S23 Ultra,31.990.000 ₫,27.190.000 ₫,-15%,201,43 đánh giá,19 thg 7 2023,chất lượng ok NHƯNG mình đề Góp Ý Nk Là khi kh...,"Nhóm Màu:Xanh Lục, Dung Lượng Lưu Trữ:8GB/256GB"


In [7]:
df1s.shape

(1582, 12)

In [8]:
df2.head(2)

,Platform_x,URL,Date Crawled,Store,Product Title,brand,Current Price,Discounted Price,Discount,Available Colors,sold_quanlity,rating_averrage,rating_count,Platform_y,customer_name,time_comment,content,thank_count,rating
0,Shopee,https://shopee.vn/Apple-iPhone-15-Plus-128GB-i...,3/19/2024,ShopDunk Official Store,Apple iPhone 15 Plus 128GB,Apple,₫29.990.000,₫23.590.000,21% GIẢM,"['Blue', 'Black', 'Pink', 'Green', 'Yellow']",354,4.9,77,Shopee,anhanh3868,2023-12-18 09:41 | Phân loại hàng: Pink,Đúng với mô tả: đúng\nTính năng nổi bật: không...,9,5
1,Shopee,https://shopee.vn/Apple-iPhone-15-Plus-128GB-i...,3/19/2024,ShopDunk Official Store,Apple iPhone 15 Plus 128GB,Apple,₫29.990.000,₫23.590.000,21% GIẢM,"['Blue', 'Black', 'Pink', 'Green', 'Yellow']",354,4.9,77,Shopee,g*****4,2024-01-17 14:14 | Phân loại hàng: Pink,Chất lượng sản phẩm: qua tuyet voi\nĐúng với m...,7,5


In [9]:
df2s = df2[['Platform_x','Date Crawled','brand','Product Title','Current Price','Discounted Price','Discount','sold_quanlity','rating_count','time_comment','content']].rename(
    columns={
        'Platform_x': 'Platform',
        'Date Crawled': 'Date_Crawl',
        'brand': 'brand',
        'Product Title': 'product_name',
        'Current Price': 'current_price',
        'Discounted Price':'discount_price' ,
        'Discount':'discount' ,
        'rating_count': 'rating_count',
        'time_comment': 'time_comment',
        'content': 'content_cmt',
    }
)

In [10]:
df2s.shape

(3126, 11)

In [11]:
df3.head(2)

,Platform,id,Date Crawled,brand_name,store,product_name,current_price,discount_price,discount,colors,...,rating_count,rating_average,comment_id,title,content,thank_count,customer_name,product_attributes,delivery_date,current_date
0,Tiki,271966786,3/11/2024,Apple,Tiki Trading,Apple iPhone 15,22490000,19490000,13,"[{'label': 'Hồng'}, {'label': 'Xanh Dương'}, {...",...,61,5.0,19623470,Cực kì hài lòng,"Tiki Giao hàng siêu nhanh, chuẩn. Giá cực tốt,...",4,Tăng Hoàng Nhân,"['Màu sắc: Xanh Dương', 'Dung lượng: 128GB']",10/31/2023 9:22,3/11/2024 11:44
1,Tiki,271966786,3/11/2024,Apple,Tiki Trading,Apple iPhone 15,22490000,19490000,13,"[{'label': 'Hồng'}, {'label': 'Xanh Dương'}, {...",...,61,5.0,19697684,Cực kì hài lòng,NaN,2,Vương Thái Dương,"['Màu sắc: Đen', 'Dung lượng: 128GB']",12/6/2023 16:35,3/11/2024 11:44


In [12]:
df3s = df3[['Platform','Date Crawled','brand_name','product_name','sold_quanlity','rating_count','delivery_date','content','product_attributes']].rename(
    columns={
        'Date Crawled': 'Date_Crawl',
        'brand_name': 'brand',
        'delivery_date': 'time_comment',
        'content': 'content_cmt',
        'product_attributes': 'info_cmt',
    }
)

In [13]:
df3s.shape

(274, 9)

In [14]:
df4.head(5)

,Platform_x,URL,Date Crawled,Product Title,Current Price,Discounted Price,Discount,Rating Average,Rating Count,Platform_y,Customer Name,Time Comment,Content
0,FPT,https://fptshop.com.vn/dien-thoai/iphone-15-pr...,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,FPT,Trung,17 ngày trước,"Dịch vụ tốt, chuyên nghiệp, máy thì đỉnh chóp"
1,FPT,https://fptshop.com.vn/dien-thoai/iphone-15-pr...,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,FPT,Đã mua tại FPTShop,NaN,NaN
2,FPT,https://fptshop.com.vn/dien-thoai/iphone-15-pr...,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,FPT,Long,Ngày 20/01/2024,"Dịch vụ tốt, máy đa dạng và đủ màu nhé"
3,FPT,https://fptshop.com.vn/dien-thoai/iphone-15-pr...,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,FPT,Đã mua tại FPTShop,NaN,NaN
4,FPT,https://fptshop.com.vn/dien-thoai/iphone-15-pr...,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,FPT,Tuấn Kiệt,Ngày 19/01/2024,Tuyệt vời


In [15]:
# Định nghĩa hàm để áp dụng logic cho việc cập nhật cột 'status'
def update_status(row, next_customer_name):
    if next_customer_name == 'Đã mua tại FPTShop':
        return 'Đã mua'
    return "None"  # Hoặc 'Chưa mua tại FPTShop' nếu bạn muốn gán giá trị này

# Tạo một cột 'next_customer_name' chứa tên khách hàng ở dòng tiếp theo
df4['next_customer_name'] = df4['Customer Name'].shift(-1)

# Áp dụng hàm update_status để tạo cột 'status'
df4['status'] = df4.apply(lambda x: update_status(x['Customer Name'], x['next_customer_name']), axis=1)

# Loại bỏ cột 'next_customer_name' vì đã hoàn thành việc sử dụng
df4 = df4.drop(columns=['next_customer_name'])

# Xóa các dòng có 'Time Comment' và 'Content' đều chứa NaN
df4 = df4.dropna(subset=['Time Comment', 'Content'], how='all')

In [16]:
df4.head(2)

,Platform_x,URL,Date Crawled,Product Title,Current Price,Discounted Price,Discount,Rating Average,Rating Count,Platform_y,Customer Name,Time Comment,Content,status
0,FPT,https://fptshop.com.vn/dien-thoai/iphone-15-pr...,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,FPT,Trung,17 ngày trước,"Dịch vụ tốt, chuyên nghiệp, máy thì đỉnh chóp",Đã mua
2,FPT,https://fptshop.com.vn/dien-thoai/iphone-15-pr...,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,FPT,Long,Ngày 20/01/2024,"Dịch vụ tốt, máy đa dạng và đủ màu nhé",Đã mua


In [17]:
df4s = df4[['Platform_x', 'Date Crawled', 'Product Title', 'Current Price', 'Discounted Price', 'Discount', 'Rating Average', 'Rating Count', 'Time Comment', 'Content', 'status']].rename(
    columns={
        'Platform_x': 'Platform',
        'Date Crawled': 'Date_Crawl',
        'Product Title': 'product_name',
        'Current Price': 'current_price',
        'Discounted Price': 'discount_price',
        'Discount': 'discount',
        'Rating Average': 'rating_average',
        'Rating Count': 'rating_count',
        'Time Comment': 'time_comment',
        'Content': 'content_cmt',
    }
)

In [18]:
df4s

,Platform,Date_Crawl,product_name,current_price,discount_price,discount,rating_average,rating_count,time_comment,content_cmt,status
0,FPT,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,17 ngày trước,"Dịch vụ tốt, chuyên nghiệp, máy thì đỉnh chóp",Đã mua
2,FPT,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,Ngày 20/01/2024,"Dịch vụ tốt, máy đa dạng và đủ màu nhé",Đã mua
4,FPT,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,Ngày 19/01/2024,Tuyệt vời,Đã mua
6,FPT,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,Ngày 19/01/2024,"Dịch vụ tốt, đc 2 năm bảo hành liền, sẽ còn ủn...",Đã mua
8,FPT,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,Ngày 13/12/2023,T,None
...,...,...,...,...,...,...,...,...,...,...,...
224634,FPT,2024-04-20,Samsung Galaxy S23 5G 256GB (No.00856552),24.990.000₫,18.990.000₫,NaN,5/5,99 đánh giá,Ngày 19/02/2023,"Chính sách cho khách khá là ưu đãi, quà tặng n...",Đã mua
224636,FPT,2024-04-20,Samsung Galaxy S23 5G 256GB (No.00856552),24.990.000₫,18.990.000₫,NaN,5/5,99 đánh giá,Ngày 19/02/2023,"Chính sách cho khách khá là ưu đãi, quà tặng n...",Đã mua
224638,FPT,2024-04-20,Samsung Galaxy S23 5G 256GB (No.00856552),24.990.000₫,18.990.000₫,NaN,5/5,99 đánh giá,Ngày 19/02/2023,"Chính sách cho khách khá là ưu đãi, quà tặng n...",Đã mua
224640,FPT,2024-04-20,Samsung Galaxy S23 5G 256GB (No.00856552),24.990.000₫,18.990.000₫,NaN,5/5,99 đánh giá,Ngày 19/02/2023,"Chính sách cho khách khá là ưu đãi, quà tặng n...",Đã mua


In [19]:
df5.head(2)

,Platform_x,URL,Date Crawled,Product Title,Current Price,Discounted Price,Discount,Rating Average,Rating Count,Platform_y,Customer Name,Time Comment,Content
0,TGDĐ,https://www.thegioididong.com/dtdd/iphone-15,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,TGDĐ,Nhựt Anh,Hữu ích Đã dùng khoảng 1 tuần,Tốt
1,TGDĐ,https://www.thegioididong.com/dtdd/iphone-15,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,TGDĐ,Nguyễn Thị Mỹ Hang,Hữu ích Đã dùng khoảng 2 tuần,Tuyết voi


In [20]:
# Thêm cột 'status' dựa vào cột 'Time Comment'
df5['status'] = df5['Time Comment'].apply(lambda x: 'Đã mua' if 'Đã dùng' in x else "None")
df5.head()

,Platform_x,URL,Date Crawled,Product Title,Current Price,Discounted Price,Discount,Rating Average,Rating Count,Platform_y,Customer Name,Time Comment,Content,status
0,TGDĐ,https://www.thegioididong.com/dtdd/iphone-15,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,TGDĐ,Nhựt Anh,Hữu ích Đã dùng khoảng 1 tuần,Tốt,Đã mua
1,TGDĐ,https://www.thegioididong.com/dtdd/iphone-15,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,TGDĐ,Nguyễn Thị Mỹ Hang,Hữu ích Đã dùng khoảng 2 tuần,Tuyết voi,Đã mua
2,TGDĐ,https://www.thegioididong.com/dtdd/iphone-15,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,TGDĐ,Ngọc Bùi,Hữu ích (5) Đã dùng khoảng 2 tháng,M muốn đổi từ 15 sang 15 plus thì ntn Máy ngoạ...,Đã mua
3,TGDĐ,https://www.thegioididong.com/dtdd/iphone-15,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,TGDĐ,Thảo Linh,Hữu ích (3) Đã dùng khoảng 3 ngày,Máy nhìn ở ngoài xinh hơn ạ. Nhân viên tư vấn ...,Đã mua
4,TGDĐ,https://www.thegioididong.com/dtdd/iphone-15,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,TGDĐ,Lâm Chấn Thịnh,Hữu ích (2) Đã dùng khoảng 1 tuần,tốt,Đã mua


In [21]:
df5s = df5[['Platform_x','Date Crawled','Product Title','Current Price','Discounted Price','Discount','Rating Average','Rating Count','Time Comment','Content','status']].rename(
    columns={
        'Platform_x': 'Platform',
        'Date Crawled': 'Date_Crawl',
        'Product Title': 'product_name',
        'Current Price': 'current_price',
        'Discounted Price': 'discount_price',
        'Discount': 'discount',
        'Rating Average': 'rating_average',
        'Rating Count': 'rating_count',
        'Time Comment': 'time_comment',
        'Content': 'content_cmt',
    }
)

In [22]:
df5s.head()

,Platform,Date_Crawl,product_name,current_price,discount_price,discount,rating_average,rating_count,time_comment,content_cmt,status
0,TGDĐ,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,Hữu ích Đã dùng khoảng 1 tuần,Tốt,Đã mua
1,TGDĐ,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,Hữu ích Đã dùng khoảng 2 tuần,Tuyết voi,Đã mua
2,TGDĐ,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,Hữu ích (5) Đã dùng khoảng 2 tháng,M muốn đổi từ 15 sang 15 plus thì ntn Máy ngoạ...,Đã mua
3,TGDĐ,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,Hữu ích (3) Đã dùng khoảng 3 ngày,Máy nhìn ở ngoài xinh hơn ạ. Nhân viên tư vấn ...,Đã mua
4,TGDĐ,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,Hữu ích (2) Đã dùng khoảng 1 tuần,tốt,Đã mua


In [23]:
df6.head(2)

,Platform_x,URL,Date Crawled,Product Title,Current Price,Discounted Price,Rating Average,Rating Count,Platform_y,Customer Name,Time Comment,Content,status,content_rate
0,CellPhones,https://cellphones.com.vn/iphone-15.html,3/10/2024,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.990.000đ,4.9/5,7 đánh giá,CellPhones,Trương Tấn Đô,31/3/2024 04:59,mong duyệt giúp em khoản góp,Đã mua tại CellphoneS,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...
1,CellPhones,https://cellphones.com.vn/iphone-15.html,3/10/2024,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.990.000đ,4.9/5,7 đánh giá,CellPhones,Mai Tùng,2/3/2024 09:39,Nhân viên bắc tư vấn nhiệt tình nhưng mua tặng...,NaN,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...


In [24]:
# Cập nhật cột 'status' dựa vào điều kiện cụ thể
df6['status'] = df6['status'].apply(lambda x: 'Đã mua' if x == 'Đã mua tại CellphoneS' else "None")

In [25]:
df6s = df6[['Platform_x','Date Crawled','Product Title','Current Price','Discounted Price','Rating Average','Rating Count','Time Comment','Content','content_rate','status']].rename(
    columns={
        'Platform_x': 'Platform',
        'Date Crawled': 'Date_Crawl',
        'Product Title': 'product_name',
        'Current Price': 'current_price',
        'Discounted Price': 'discount_price',
        'Rating Average': 'rating_average',
        'Rating Count': 'rating_count',
        'Time Comment': 'time_comment',
        'Content': 'content_cmt',
    }
)

In [26]:
df6s.head()

,Platform,Date_Crawl,product_name,current_price,discount_price,rating_average,rating_count,time_comment,content_cmt,content_rate,status
0,CellPhones,3/10/2024,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.990.000đ,4.9/5,7 đánh giá,31/3/2024 04:59,mong duyệt giúp em khoản góp,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,Đã mua
1,CellPhones,3/10/2024,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.990.000đ,4.9/5,7 đánh giá,2/3/2024 09:39,Nhân viên bắc tư vấn nhiệt tình nhưng mua tặng...,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,None
2,CellPhones,3/10/2024,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.990.000đ,4.9/5,7 đánh giá,7/11/2023 17:04,Tần số quét\n60 Hz\nĐộ sáng tối đa\n2000 nits\...,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,None
3,CellPhones,3/10/2024,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.990.000đ,4.9/5,7 đánh giá,7/11/2023 02:19,Awesome phone! Quá tốt,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,None
4,CellPhones,3/10/2024,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.990.000đ,4.9/5,7 đánh giá,7/11/2023 02:12,mọi thử ổn trừ camera trước,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,None


### Lazada

In [27]:
df1s.head(2)

,Platform,Date_Crawl,brand,product_name,current_price,discount_price,discount,sold_quanlity,rating_count,time_comment,content_cmt,info_cmt
0,Lazada,3/13/2024,Apple,iPhone 15 - Hàng Chính Hãng VN/A,25.990.000 ₫,24.690.000 ₫,-5%,707 Đã bán,147 đánh giá,14 thg 11 2023,Mua hàng chính hãng tại Apple flagship store t...,"Nhóm Màu:Pink, Dung Lượng Lưu Trữ:128GB"
1,Lazada,3/13/2024,Apple,iPhone 15 - Hàng Chính Hãng VN/A,25.990.000 ₫,24.690.000 ₫,-5%,707 Đã bán,147 đánh giá,02 thg 12 2023,Chất lượng: \n- Giao hàng nhanh. \n- SP đúng ...,"Nhóm Màu:Xanh dương, Dung Lượng Lưu Trữ:128GB"


In [28]:
def extract_color_and_capacy(df, column_name):
    # Chuẩn hóa và tách thông tin từ cột
    def normalize_and_extract(info):
        # Chuyển định dạng danh sách thành chuỗi
        if info.startswith('[') and info.endswith(']'):
            info = re.sub(r"[\[\]']", "", info)
        # Trích xuất thông tin màu sắc và dung lượng
        color_match = re.search(r"(Nhóm Màu|Màu sắc): ?([^,]+)", info)
        capacy_match = re.search(r"(Dung Lượng Lưu Trữ|Dung lượng): ?([^,]+)", info)
        color = color_match.group(2) if color_match else None
        capacy = capacy_match.group(2) if capacy_match else None
        return pd.Series([color, capacy])

    # Áp dụng hàm trên cho mỗi hàng và tạo cột mới
    df[['color', 'capacy']] = df[column_name].apply(normalize_and_extract)
    
    # Xóa cột chứa thông tin gốc
    df = df.drop(columns=[column_name])
    
    return df

def extract_product_name(df, column_name):
    def clean_and_refine_name(name):

        name = name.replace("Điện thoại ", "")
        
        name = re.sub(r' - Hàng chính hãng.*$', '', name)
        
        name = re.sub(r'\s*\([^)]+\)', '', name)
        
        primary_name = name.split(' -')[0]
        
        primary_name = re.sub(r'(\d+GB|Chính hãng).*', '', primary_name).strip()
        
        if 'iPhone 15' in primary_name and not primary_name.startswith('Apple'):
            primary_name = 'Apple ' + primary_name
        
        # Ensure we don't prepend 'Apple' to non-iPhone products
        elif 'iPhone' not in primary_name and primary_name.startswith('Apple'):
            primary_name = primary_name.replace('Apple ', '', 1)
        
        return primary_name.strip()  # Trim any whitespace from the ends

    # Apply the cleaning and refining function to each product name
    df[column_name] = df[column_name].apply(clean_and_refine_name)

    return df

def convert_time(value):
    # Đầu tiên, kiểm tra xem giá trị có phải là chuỗi không
    if not isinstance(value, str):
        # Nếu không phải chuỗi, kiểm tra xem đã là datetime chưa
        if isinstance(value, datetime):
            # Nếu đã là datetime, trả về nguyên vẹn
            return value
        else:
            # Nếu không phải chuỗi hoặc datetime, có thể chuyển thành chuỗi hoặc xử lý khác
            return None

    # Xử lý giá trị đặc biệt như "17 giờ trước"
    if 'giờ trước' in value:
        hours_ago = int(value.split(' ')[0])
        date_obj = datetime.now() - timedelta(hours=hours_ago)
    else:
        try:
            if 'thg' in value:
                date_obj = datetime.strptime(value, "%d thg %m %Y")
            else:
                date_obj = datetime.strptime(value, "%m/%d/%Y")
        except ValueError:
            # Trường hợp không thể chuyển đổi, trả về None
            return None
    # Định dạng datetime theo yêu cầu
    return date_obj

def convert_column_to_datetime(df, column_name):
    df[column_name] = df[column_name].apply(convert_time)
    # Sử dụng errors='coerce' để chuyển những giá trị không hợp lệ thành NaT
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    return df

def extract_numbers(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: int(re.findall(r'\d+', str(x))[0]) if re.findall(r'\d+', str(x)) else 0)
    
    return df

In [29]:
df1s = extract_product_name(df1s, 'product_name')
# Sử dụng hàm trên với DataFrame df1s và cột 'info_cmt'
df1s = convert_column_to_datetime(df1s, 'time_comment')
df1s = convert_column_to_datetime(df1s, 'Date_Crawl')
df1s = extract_color_and_capacy(df1s, 'info_cmt')
df1s = extract_numbers(df1s, 'sold_quanlity')
df1s = extract_numbers(df1s, 'rating_count')
df1s['Date_Crawl'] = pd.to_datetime(df1s['Date_Crawl'], format='%Y-%m-%d', errors='coerce')
df1s['time_comment'] = pd.to_datetime(df1s['time_comment'], format='%Y-%m-%d', errors='coerce')

In [30]:
df1s.head(2)

,Platform,Date_Crawl,brand,product_name,current_price,discount_price,discount,sold_quanlity,rating_count,time_comment,content_cmt,color,capacy
0,Lazada,2024-03-13,Apple,Apple iPhone 15,25.990.000 ₫,24.690.000 ₫,-5%,707,147,2023-11-14,Mua hàng chính hãng tại Apple flagship store t...,Pink,128GB
1,Lazada,2024-03-13,Apple,Apple iPhone 15,25.990.000 ₫,24.690.000 ₫,-5%,707,147,2023-12-02,Chất lượng: \n- Giao hàng nhanh. \n- SP đúng ...,Xanh dương,128GB


### Shopee

In [31]:
df2s.head(2)

,Platform,Date_Crawl,brand,product_name,current_price,discount_price,discount,sold_quanlity,rating_count,time_comment,content_cmt
0,Shopee,3/19/2024,Apple,Apple iPhone 15 Plus 128GB,₫29.990.000,₫23.590.000,21% GIẢM,354,77,2023-12-18 09:41 | Phân loại hàng: Pink,Đúng với mô tả: đúng\nTính năng nổi bật: không...
1,Shopee,3/19/2024,Apple,Apple iPhone 15 Plus 128GB,₫29.990.000,₫23.590.000,21% GIẢM,354,77,2024-01-17 14:14 | Phân loại hàng: Pink,Chất lượng sản phẩm: qua tuyet voi\nĐúng với m...


In [32]:
def split_time_comment_and_color(df, column_name):
    # Đảm bảo cột là chuỗi
    df[column_name] = df[column_name].astype(str)

    # Tách và tạo cột 'color'
    df['color'] = df[column_name].apply(lambda x: x.split('| Phân loại hàng: ')[1] if '| Phân loại hàng: ' in x else None)
    
    # Tách và cập nhật cột 'time_comment'
    df['time_comment'] = df[column_name].apply(lambda x: x.split(' |')[0] if ' |' in x else x)

    return df

def extract_date_from_datetime(df, column_name):
    # Chuyển đổi cột sang định dạng datetime (nếu chưa là datetime)
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    
    # Chỉ giữ lại phần ngày từ cột datetime
    df[column_name] = df[column_name].dt.date
    
    return df

def convert_k_to_thousand(value):
    value = str(value).lower()
    if 'k' in value:
        # Loại bỏ ký tự 'k', chuyển đổi sang số, sau đó nhân với 1000
        return int(float(value.replace('k', '')) * 1000)
    else:
        try:
            # Trả về giá trị số như ban đầu, chuyển về số nguyên để loại bỏ .0
            return int(float(value.replace(',', '.')))
        except ValueError:
            # Trường hợp không thể chuyển đổi, trả về None hoặc 0
            return None

def convert_k_to_thousand(value):
    # Chuyển đổi dấu phẩy thành dấu chấm để xử lý đúng định dạng số thực
    value = str(value).replace('k', '').replace(',', '.').strip()
    
    # Kiểm tra và chuyển đổi giá trị
    if value:  # Kiểm tra xem chuỗi sau khi bỏ 'k' và khoảng trắng có trống không
        try:
            return int(float(value) * 1000)
        except ValueError:
            # Trường hợp không thể chuyển đổi, trả về None hoặc 0 hoặc giữ nguyên giá trị
            # Tùy thuộc vào cách bạn muốn xử lý lỗi này
            return None
    else:
        return None

def convert_rating_count(df, column_name):
    df[column_name] = df[column_name].apply(convert_k_to_thousand)
    return df

def split_product_name_and_capacity(df):
    # Ensure the column is of string type
    df['product_name'] = df['product_name'].astype(str)
    
    # Regular expression to search for capacity information
    capacity_pattern = re.compile(r'(\d+GB|\d+TB)')
    
    # Extract capacity information and update the product name
    df['capacy'] = df['product_name'].apply(lambda x: capacity_pattern.search(x).group(1) if capacity_pattern.search(x) else None)
    df['product_name'] = df['product_name'].str.replace(capacity_pattern, '', regex=True).str.strip()
    
    # Remove the term "Điện thoại" or "Điện Thoại" if present, using case-insensitive replacement
    df['product_name'] = df['product_name'].str.replace(r"(?i)điện thoại\s*", "", regex=True).str.strip()
    
    # Remove any text after (and including) a dash '-'
    df['product_name'] = df['product_name'].apply(lambda x: re.split(r'\s*-\s*', x)[0].strip())

    # Remove additional specified text
    df['product_name'] = df['product_name'].str.replace('Chính hãng VN/A', '', regex=False).str.strip()
    df['product_name'] = df['product_name'].str.replace('Hàng chính hãng mới 100% New Seal', '', regex=False).str.strip()
    
    # Directly convert "Samsung Galaxy S23+" to "Samsung Galaxy S23 Plus"
    df['product_name'] = df['product_name'].str.replace(r"Samsung Galaxy S23\+", "Samsung Galaxy S23 Plus", regex=True)
    
    # Standardize variations of "Samsung Galaxy S23 Plus" and remove parentheses
    df['product_name'] = df['product_name'].str.replace(r"(Samsung Galaxy S23 Plus)\s*\(?S23\+\)?", r"\1", regex=True)
    df['product_name'] = df['product_name'].str.replace(r"(Samsung Galaxy S23)\s*Plus\s*\(?S23\+\)?", r"\1 Plus", regex=True)

    # Remove any content within parentheses across all product names
    df['product_name'] = df['product_name'].str.replace(r"\s*\([^)]*\)", "", regex=True).str.strip()

    return df

def split_color_and_capacity(df, color_column):
    # Compile a regex pattern to identify capacity information
    capacity_pattern = re.compile(r'\b(\d+GB|\d+TB)\b')
    
    # Ensure there's a 'capacity' column, fill with None if doesn't exist
    if 'capacy' not in df.columns:  # Corrected typo from 'capacy' to 'capacity'
        df['capacy'] = None
    
    # Define a function to extract capacity from the mixed data
    def extract_capacity(text, existing_capacity):
        if pd.notna(existing_capacity):  # If there's already a capacity value, keep it
            return existing_capacity
        match = capacity_pattern.search(text)
        return match.group(1) if match else existing_capacity
    
    # Define a function to remove capacity info from the mixed data, leaving only color
    def remove_capacity(text):
        # Remove capacity information
        new_text = capacity_pattern.sub('', text).strip()
        # Remove any remaining digits followed by "/" which are not related to color
        new_text = re.sub(r',\s*\d+/', '', new_text).strip(', ')
        return new_text
    
    df['capacy'] = df.apply(lambda x: extract_capacity(x[color_column], x['capacy']), axis=1)
    df[color_column] = df[color_column].apply(remove_capacity)
    
    return df

In [33]:
df2s = split_time_comment_and_color(df2s, 'time_comment')
df2s = extract_date_from_datetime(df2s, 'time_comment')
df2s = convert_column_to_datetime(df2s, 'Date_Crawl')
df2s = convert_rating_count(df2s, 'rating_count')
df2s = split_product_name_and_capacity(df2s)
df2s['Date_Crawl'] = pd.to_datetime(df2s['Date_Crawl'], format='%Y-%m-%d', errors='coerce')
df2s['time_comment'] = pd.to_datetime(df2s['time_comment'], format='%Y-%m-%d', errors='coerce')
df2s = split_color_and_capacity(df2s, 'color')

In [34]:
df2s.head()

,Platform,Date_Crawl,brand,product_name,current_price,discount_price,discount,sold_quanlity,rating_count,time_comment,content_cmt,color,capacy
0,Shopee,2024-03-19,Apple,Apple iPhone 15 Plus,₫29.990.000,₫23.590.000,21% GIẢM,354,77000,2023-12-18,Đúng với mô tả: đúng\nTính năng nổi bật: không...,Pink,128GB
1,Shopee,2024-03-19,Apple,Apple iPhone 15 Plus,₫29.990.000,₫23.590.000,21% GIẢM,354,77000,2024-01-17,Chất lượng sản phẩm: qua tuyet voi\nĐúng với m...,Pink,128GB
2,Shopee,2024-03-19,Apple,Apple iPhone 15 Plus,₫29.990.000,₫23.590.000,21% GIẢM,354,77000,2024-01-30,"Đã nhận hàng, đóng gói rất cẩn thận. Ip thì rấ...",Green,128GB
3,Shopee,2024-03-19,Apple,Apple iPhone 15 Plus,₫29.990.000,₫23.590.000,21% GIẢM,354,77000,2023-10-22,Thật là tuyệt vời khi nhận được em nó. Màu đẹp...,Pink,128GB
4,Shopee,2024-03-19,Apple,Apple iPhone 15 Plus,₫29.990.000,₫23.590.000,21% GIẢM,354,77000,2024-03-09,Rất hài lòng về chất lượng máy và cách đóng gó...,Green,128GB


### Tiki

In [35]:
df3s.head(2)

,Platform,Date_Crawl,brand,product_name,sold_quanlity,rating_count,time_comment,content_cmt,info_cmt
0,Tiki,3/11/2024,Apple,Apple iPhone 15,380,61,10/31/2023 9:22,"Tiki Giao hàng siêu nhanh, chuẩn. Giá cực tốt,...","['Màu sắc: Xanh Dương', 'Dung lượng: 128GB']"
1,Tiki,3/11/2024,Apple,Apple iPhone 15,380,61,12/6/2023 16:35,NaN,"['Màu sắc: Đen', 'Dung lượng: 128GB']"


In [36]:
df3s = extract_date_from_datetime(df3s, 'time_comment')
df3s = convert_column_to_datetime(df3s, 'Date_Crawl')
df3s = extract_color_and_capacy(df3s, 'info_cmt')
df3s['Date_Crawl'] = pd.to_datetime(df3s['Date_Crawl'], format='%Y-%m-%d', errors='coerce')
df3s['time_comment'] = pd.to_datetime(df3s['time_comment'], format='%Y-%m-%d', errors='coerce')

In [37]:
df3s.head()

,Platform,Date_Crawl,brand,product_name,sold_quanlity,rating_count,time_comment,content_cmt,color,capacy
0,Tiki,2024-03-11,Apple,Apple iPhone 15,380,61,2023-10-31,"Tiki Giao hàng siêu nhanh, chuẩn. Giá cực tốt,...",Xanh Dương,128GB
1,Tiki,2024-03-11,Apple,Apple iPhone 15,380,61,2023-12-06,NaN,Đen,128GB
2,Tiki,2024-03-11,Apple,Apple iPhone 15,380,61,2024-03-06,Máy giao đến sau 3 ngày (dự kiến lúc đầu là 7 ...,Hồng,128GB
3,Tiki,2024-03-11,Apple,Apple iPhone 15,380,61,2023-11-27,Tốt nhất là các bác nên ra ĐMX hoặc TGDD mà mu...,Đen,256GB
4,Tiki,2024-03-11,Apple,Apple iPhone 15,380,61,2023-12-15,Đóng gói chắc chắn. Giao hàng nhanh. Nguyên se...,Hồng,128GB


### FPT Shop

In [38]:
df4s.head(2)

,Platform,Date_Crawl,product_name,current_price,discount_price,discount,rating_average,rating_count,time_comment,content_cmt,status
0,FPT,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,17 ngày trước,"Dịch vụ tốt, chuyên nghiệp, máy thì đỉnh chóp",Đã mua
2,FPT,3/9/2024,iPhone 15 Pro Max 256GB (No.00904369),34.990.000₫,30.990.000₫,-11%,5-Apr,63 đánh giá,Ngày 20/01/2024,"Dịch vụ tốt, máy đa dạng và đủ màu nhé",Đã mua


In [39]:
def remove_currency_symbol(df, column_name):
    # Thay thế ký tự tiền tệ (₫) bằng chuỗi rỗng
    df[column_name] = df[column_name].str.replace('₫', '', regex=False)
    
    return df

def convert_discount(df, column_name):
    # Loại bỏ dấu '-' và giữ lại ký tự '%'
    df[column_name] = df[column_name].str.replace('-', '', regex=False)
    
    return df

def extract_before_slash(df, column_name):
    # Chỉ giữ lại phần trước dấu '/' trong mỗi giá trị của cột
    df[column_name] = df[column_name].apply(lambda x: str(x).split('/')[0] if '/' in str(x) else x)
    
    return df

def extract_capacity_and_update_product_name(df, column_name):
    # Đảm bảo cột tồn tại trong DataFrame
    if column_name in df.columns:
        # Tạo biểu thức chính quy để tìm kiếm thông tin dung lượng (ví dụ: '128GB' hoặc '1TB')
        capacity_pattern = re.compile(r'(\d+(GB|TB))')

        # Trích xuất thông tin dung lượng và lưu vào cột mới 'capacity'
        df['capacity'] = df[column_name].astype(str).apply(lambda x: capacity_pattern.search(x).group(1) if capacity_pattern.search(x) else 'Unknown')

        # Loại bỏ thông tin dung lượng từ 'product_name' và bất kỳ thông tin phụ nào sau đó
        df[column_name] = df[column_name].astype(str).apply(lambda x: re.sub(r'\s*\d+(GB|TB).*', '', x))

        # Loại bỏ từ "Điện thoại" từ 'product_name' nếu có
        df[column_name] = df[column_name].apply(lambda x: x.replace("Điện thoại ", "").strip())

    return df

def convert_to_absolute_date(df, column_name):
    def convert_date(value):
        # Kiểm tra giá trị NaN và bỏ qua xử lý nếu đúng
        if pd.isnull(value):
            return value
        
        today = datetime.now()
        if "ngày trước" in value:
            num = int(value.split()[0])
            # Trả về ngày sau khi trừ đi số ngày tương ứng, loại bỏ thông tin thời gian
            return (today - timedelta(days=num)).date()
        elif "Ngày" in value:
            # Xử lý dữ liệu cụ thể như 'Ngày 20/01/2024', trả về ngày không kèm thời gian
            date_obj = datetime.strptime(value, 'Ngày %d/%m/%Y')
            return date_obj.date()
        else:
            # Nếu không khớp với bất kỳ định dạng nào, giữ nguyên giá trị
            return value
    
    # Áp dụng hàm chuyển đổi
    df[column_name] = df[column_name].apply(convert_date)
    
    return df

In [40]:
df4s = extract_numbers(df4s, 'rating_count')
df4s = convert_discount(df4s, 'discount')
df4s = remove_currency_symbol(df4s, 'current_price')
df4s = remove_currency_symbol(df4s, 'discount_price')
df4s = extract_before_slash(df4s, 'rating_average')
df4s = extract_capacity_and_update_product_name(df4s, 'product_name')
df4s = convert_to_absolute_date(df4s, 'time_comment')
df4s['Date_Crawl'] = pd.to_datetime(df4s['Date_Crawl'], format='%m/%d/%Y', errors='coerce')

In [41]:
df4s.head()

,Platform,Date_Crawl,product_name,current_price,discount_price,discount,rating_average,rating_count,time_comment,content_cmt,status,capacity
0,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2024-04-04,"Dịch vụ tốt, chuyên nghiệp, máy thì đỉnh chóp",Đã mua,256GB
2,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2024-01-20,"Dịch vụ tốt, máy đa dạng và đủ màu nhé",Đã mua,256GB
4,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2024-01-19,Tuyệt vời,Đã mua,256GB
6,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2024-01-19,"Dịch vụ tốt, đc 2 năm bảo hành liền, sẽ còn ủn...",Đã mua,256GB
8,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2023-12-13,T,None,256GB


### Thế Giới Di Động

In [42]:
df5s.head(2)

,Platform,Date_Crawl,product_name,current_price,discount_price,discount,rating_average,rating_count,time_comment,content_cmt,status
0,TGDĐ,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,Hữu ích Đã dùng khoảng 1 tuần,Tốt,Đã mua
1,TGDĐ,3/10/2024,Điện thoại iPhone 15 128GB,22.990.000₫,20.690.000₫,-10%,4.5,14 đánh giá,Hữu ích Đã dùng khoảng 2 tuần,Tuyết voi,Đã mua


In [43]:
def remove_useful_prefix(df, column_name):
    # Loại bỏ từ "Hữu ích" và dấu cách đằng sau (nếu có) bằng cách thay thế bằng chuỗi rỗng
    df[column_name] = df[column_name].str.replace('Hữu ích ', '', regex=False)
    df[column_name] = df[column_name].str.replace(r'\(.*?\)', '', regex=True).str.strip()

    return df

In [44]:
df5s = extract_numbers(df5s, 'rating_count')
df5s = convert_discount(df5s, 'discount')
df5s = remove_currency_symbol(df5s, 'current_price')
df5s = remove_currency_symbol(df5s, 'discount_price')
df5s = extract_capacity_and_update_product_name(df5s, 'product_name')
df5s = remove_useful_prefix(df5s, 'time_comment')
df5s['Date_Crawl'] = pd.to_datetime(df5s['Date_Crawl'], format='%m/%d/%Y', errors='coerce')

In [45]:
df5s.head()

,Platform,Date_Crawl,product_name,current_price,discount_price,discount,rating_average,rating_count,time_comment,content_cmt,status,capacity
0,TGDĐ,2024-03-10,iPhone 15,22.990.000,20.690.000,10%,4.5,14,Đã dùng khoảng 1 tuần,Tốt,Đã mua,128GB
1,TGDĐ,2024-03-10,iPhone 15,22.990.000,20.690.000,10%,4.5,14,Đã dùng khoảng 2 tuần,Tuyết voi,Đã mua,128GB
2,TGDĐ,2024-03-10,iPhone 15,22.990.000,20.690.000,10%,4.5,14,Đã dùng khoảng 2 tháng,M muốn đổi từ 15 sang 15 plus thì ntn Máy ngoạ...,Đã mua,128GB
3,TGDĐ,2024-03-10,iPhone 15,22.990.000,20.690.000,10%,4.5,14,Đã dùng khoảng 3 ngày,Máy nhìn ở ngoài xinh hơn ạ. Nhân viên tư vấn ...,Đã mua,128GB
4,TGDĐ,2024-03-10,iPhone 15,22.990.000,20.690.000,10%,4.5,14,Đã dùng khoảng 1 tuần,tốt,Đã mua,128GB


### CellPhones

In [46]:
df6s.head(2)

,Platform,Date_Crawl,product_name,current_price,discount_price,rating_average,rating_count,time_comment,content_cmt,content_rate,status
0,CellPhones,3/10/2024,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.990.000đ,4.9/5,7 đánh giá,31/3/2024 04:59,mong duyệt giúp em khoản góp,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,Đã mua
1,CellPhones,3/10/2024,iPhone 15 128GB | Chính hãng VN/A,22.990.000đ,19.990.000đ,4.9/5,7 đánh giá,2/3/2024 09:39,Nhân viên bắc tư vấn nhiệt tình nhưng mua tặng...,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,None


In [47]:
df6s = extract_numbers(df6s, 'rating_count')
df6s = extract_before_slash(df6s, 'rating_average')
df6s = remove_currency_symbol(df6s, 'current_price')
df6s = remove_currency_symbol(df6s, 'discount_price')
df6s = extract_date_from_datetime(df6s, 'time_comment')
df6s = convert_column_to_datetime(df6s, 'Date_Crawl')
df6s = extract_capacity_and_update_product_name(df6s, 'product_name')

C:\Users\HELLO\AppData\Local\Temp\ipykernel_11384\1406430958.py:15: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[column_name] = pd.to_datetime(df[column_name], errors='coerce')


In [48]:
df6s.head()

,Platform,Date_Crawl,product_name,current_price,discount_price,rating_average,rating_count,time_comment,content_cmt,content_rate,status,capacity
0,CellPhones,2024-03-10,iPhone 15,22.990.000đ,19.990.000đ,4.9,7,2024-03-31,mong duyệt giúp em khoản góp,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,Đã mua,128GB
1,CellPhones,2024-03-10,iPhone 15,22.990.000đ,19.990.000đ,4.9,7,2024-03-02,Nhân viên bắc tư vấn nhiệt tình nhưng mua tặng...,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,None,128GB
2,CellPhones,2024-03-10,iPhone 15,22.990.000đ,19.990.000đ,4.9,7,2023-11-07,Tần số quét\n60 Hz\nĐộ sáng tối đa\n2000 nits\...,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,None,128GB
3,CellPhones,2024-03-10,iPhone 15,22.990.000đ,19.990.000đ,4.9,7,2023-11-07,Awesome phone! Quá tốt,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,None,128GB
4,CellPhones,2024-03-10,iPhone 15,22.990.000đ,19.990.000đ,4.9,7,2023-11-07,mọi thử ổn trừ camera trước,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...,None,128GB


# 2. Tích hợp dữ liệu

In [49]:
print(df1s.shape,df2s.shape,df3s.shape,df4s.shape,df5s.shape,df6s.shape)

(1582, 13) (3126, 13) (274, 10) (128509, 12) (10902, 12) (1769, 12)


In [50]:
def combine_dataframes_vertically(*dataframes):
    combined_df = pd.concat(dataframes, ignore_index=True, axis=0, join='outer')
    # Đảm bảo rằng cột 'time_comment' được chuyển đổi sang datetime nếu chưa phải
    combined_df['time_comment'] = pd.to_datetime(combined_df['time_comment'], errors='coerce')
    # Chỉ giữ lại phần ngày trong cột 'time_comment'
    combined_df['time_comment'] = combined_df['time_comment'].dt.date
    return combined_df

In [51]:
ecommerce_data = combine_dataframes_vertically(df1s,df2s,df3s)
sales_store_data = combine_dataframes_vertically(df4s,df5s,df6s)

In [52]:
ecommerce_data.shape

(4982, 13)

In [53]:
ecommerce_data

,Platform,Date_Crawl,brand,product_name,current_price,discount_price,discount,sold_quanlity,rating_count,time_comment,content_cmt,color,capacy
0,Lazada,2024-03-13,Apple,Apple iPhone 15,25.990.000 ₫,24.690.000 ₫,-5%,707,147,2023-11-14,Mua hàng chính hãng tại Apple flagship store t...,Pink,128GB
1,Lazada,2024-03-13,Apple,Apple iPhone 15,25.990.000 ₫,24.690.000 ₫,-5%,707,147,2023-12-02,Chất lượng: \n- Giao hàng nhanh. \n- SP đúng ...,Xanh dương,128GB
2,Lazada,2024-03-13,Apple,Apple iPhone 15,25.990.000 ₫,24.690.000 ₫,-5%,707,147,2024-04-21,"Shop chuẩn bị giao hàng nhanh, hình ảnh mang t...",Black,128GB
3,Lazada,2024-03-13,Apple,Apple iPhone 15,25.990.000 ₫,24.690.000 ₫,-5%,707,147,2023-10-17,"Giao hàng đúng hẹn, chất lượng máy tốt",Màu Xanh Lá Cây,256GB
4,Lazada,2024-03-13,Apple,Apple iPhone 15,25.990.000 ₫,24.690.000 ₫,-5%,707,147,2023-09-30,"Màu hồng cưng xiu, triệu like",Hồng,256GB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4977,Tiki,2024-03-11,Apple,Apple iPhone 15 Pro,NaN,NaN,NaN,319,68,2023-10-09,NaN,Titan Xanh,128GB
4978,Tiki,2024-03-11,Apple,Apple iPhone 15 Pro,NaN,NaN,NaN,319,68,2023-10-07,NaN,Titan Tự Nhiên,128GB
4979,Tiki,2024-03-11,Apple,Apple iPhone 15 Pro,NaN,NaN,NaN,319,68,2023-10-06,NaN,Titan Tự Nhiên,256GB
4980,Tiki,2024-03-11,Apple,Apple iPhone 15 Pro,NaN,NaN,NaN,319,68,2023-10-02,NaN,Titan Xanh,256GB


In [54]:
sales_store_data.shape

(141180, 13)

In [55]:
sales_store_data

,Platform,Date_Crawl,product_name,current_price,discount_price,discount,rating_average,rating_count,time_comment,content_cmt,status,capacity,content_rate
0,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2024-04-04,"Dịch vụ tốt, chuyên nghiệp, máy thì đỉnh chóp",Đã mua,256GB,NaN
1,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2024-01-20,"Dịch vụ tốt, máy đa dạng và đủ màu nhé",Đã mua,256GB,NaN
2,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2024-01-19,Tuyệt vời,Đã mua,256GB,NaN
3,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2024-01-19,"Dịch vụ tốt, đc 2 năm bảo hành liền, sẽ còn ủn...",Đã mua,256GB,NaN
4,FPT,2024-03-09,iPhone 15 Pro Max,34.990.000,30.990.000,11%,5-Apr,63,2023-12-13,T,None,256GB,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141175,CellPhones,NaT,Samsung Galaxy S23 Ultra,31.990.000,20.990.000đ,NaN,4.9,19,2023-10-06,Quá đẹp và mãn nhãn,None,256GB,Hiệu năng Giật lag\nThời lượng pin Rất yếu\nCh...
141176,CellPhones,NaT,Samsung Galaxy S23 Ultra,31.990.000,20.990.000đ,NaN,4.9,19,2023-10-06,Đẹp mà trả góp dễ,None,256GB,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...
141177,CellPhones,NaT,Samsung Galaxy S23 Ultra,36.990.000đ,24.990.000đ,NaN,5.0,2,2023-10-11,Nghiệp vụ nhân viên bán hàng rất tôt,None,12GB,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...
141178,CellPhones,NaT,Samsung Galaxy S23 Ultra,36.990.000đ,24.990.000đ,NaN,5.0,2,2023-09-07,Quá xịn trong tầm giá haha,None,12GB,Hiệu năng Siêu mạnh mẽ\nThời lượng pin Cực khủ...


In [56]:
ecommerce_data.to_csv('Smarphones_TMĐT.csv', mode='a', header=not os.path.exists('Smarphones_TMĐT.csv'), index=False, encoding='utf-8-sig')
sales_store_data.to_csv('Smarphones_Shop.csv', mode='a', header=not os.path.exists('Smarphones_Shop.csv'), index=False, encoding='utf-8-sig')